In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import imutils

In [10]:
# function to detect motion 
def detectMotion(frame, gray, prevFrame):
    motion = cv2.absdiff(prevFrame, gray) 
    
    th = cv2.threshold(motion, 10, 255, cv2.THRESH_BINARY)[1]
    th = cv2.dilate(th, None, iterations=2) # without, threshold image has "only" serperate segments
    
    ct = cv2.findContours(th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ct = imutils.grab_contours(ct)
    
    for i in ct:
        a = cv2.contourArea(i)
        #print(a) # 'debug' solution
        if a < minArea:
            continue
            
        (x,y,w,h) = cv2.boundingRect(i)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255,0), 2)
    
    return frame, motion, th;

In [12]:
# only works properly if camera isnt moving
prevFrame = None
minArea = 1000
video = cv2.VideoCapture(0) # 0=webcam feed, or insert video of your choice: e.g. 'video.mp4' 

while video.isOpened():
    retval, frame = video.read()
    if not retval: 
        print("Video ended")
        break
    # resize video (every single frame)
    percent = 0.7
    width  = int(frame.shape[1])
    height = int(frame.shape[0])
    dim = (int(width * percent), int(height * percent))
    frame = cv2.resize(frame, dim)

    # alternative solution
    # resized = imutils.resize(frame, width = 500)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11,11), 0)
    
    if prevFrame is None:
        prevFrame = gray
        continue
    
    frame, motion, th = detectMotion(frame, gray, prevFrame)       
    
    cv2.imshow('out_cv', motion)
    #cv2.imshow('edges', edges)
    cv2.imshow('frame', frame)
    cv2.imshow('th', th)
    cv2.imshow('gray', gray)
    
    if cv2.waitKey(1) == 13:
        break
        
    prevFrame = gray

video.release()
cv2.destroyAllWindows()